In [66]:
from thumbSortToJSON import *
from sortThumbs import *
import json
from math import *
from numpy import *
from operator import itemgetter
import json
import os
import collections

class OrderedSet(collections.MutableSet):

    def __init__(self, iterable=None):
        self.end = end = [] 
        end += [None, end, end]         # sentinel node for doubly linked list
        self.map = {}                   # key --> [key, prev, next]
        if iterable is not None:
            self |= iterable

    def __len__(self):
        return len(self.map)

    def __contains__(self, key):
        return key in self.map

    def add(self, key):
        if key not in self.map:
            end = self.end
            curr = end[1]
            curr[2] = end[1] = self.map[key] = [key, curr, end]

    def discard(self, key):
        if key in self.map:        
            key, prev, next = self.map.pop(key)
            prev[2] = next
            next[1] = prev

    def __iter__(self):
        end = self.end
        curr = end[2]
        while curr is not end:
            yield curr[0]
            curr = curr[2]

    def __reversed__(self):
        end = self.end
        curr = end[1]
        while curr is not end:
            yield curr[0]
            curr = curr[1]

    def pop(self, last=True):
        if not self:
            raise KeyError('set is empty')
        key = self.end[1][0] if last else self.end[2][0]
        self.discard(key)
        return key

    def __repr__(self):
        if not self:
            return '%s()' % (self.__class__.__name__,)
        return '%s(%r)' % (self.__class__.__name__, list(self))

    def __eq__(self, other):
        if isinstance(other, OrderedSet):
            return len(self) == len(other) and list(self) == list(other)
        return set(self) == set(other)

In [49]:
fp = open(r'D:\memeproject\thumbsnailSummaryTitlesAll','r')
allThumbs = pickle.load(fp)
fp.close()

Verify that a js file is correct
==

In [218]:
def verifyMemes(name):
    memes = json.load(open("D:\\memeproject\\code\\html\\memes\\" + name,'r'))
    n = int(ceil(sqrt(len(memes))))
    print "Meme size is " + str(len(memes))
    clickedList = previewThumbs([meme['id'] + '.jpg' for meme in memes], (n,n))
    show()
    return clickedList, memes

In [219]:
name = "Insanity_Wolf.js"
clickedList, memes = verifyMemes(name) # memes[clickedList[0]] returns the first clicked item

Meme size is 2434


In [220]:
selectedMemes = itemgetter(*clickedList)([meme["id"] + ".jpg" for meme in memes])
js = toJSON(allThumbs, selectedMemes)
fp = open('D:\\memeproject\\code\html\\memes\\' + name, 'w')
json.dump(js, fp)
fp.close()

Verify Entire Directory is ok
--

If no thumbnail is selected, nothing happens

In [40]:
allMemes = os.listdir("D:\\memeproject\\code\html\\memes\\")
for name in allMemes[0:1]:
    clickedList,memes = verifyMemes(name)
    # If nothing is selected, assume that there are no errors
    if len(clickedList) != 0:
        thumbList = [meme["id"] + ".jpg" for meme in memes]
        selectedMemes = itemgetter(*clickedList)(thumbList)
        fp = open('D:\\memeproject\\code\html\\memes\\' + name, 'w')
        json.dump(toJSON(allThumbs, selectedMemes), fp)
        fp.close()
        print "Saved" + name
    else:
        print "Passed" + name

Passed3rd_World_Success_Kid.js


Compare Thumbnails with a picture
==

1) Load Thumbnail and generate thumbList, a sorted list of differences

2) Use GUI to determine "Clicked List"

3) Finally save the json file. If a new template is generated, templatesToJSON.py might have to be invoked

In [225]:
%%capture
loc = r"D:\memeproject\templates\Awkward_Penguin.jpg"
#loc = r"D:\memeproject\thumbnails\11hb7w.jpg"
(thumbList, a) = compareAll(loc, allThumbs)

In [223]:
offset = 3000
clickedList = previewThumbs([thumb[1] for thumb in thumbList][offset:], (50,100))
show()
clickedList = range(0,offset) + [i + offset for i in clickedList]

In [224]:
name = "Awkward_Penguin"
selectedMemes = itemgetter(*clickedList)(thumbList)
fp = open('D:\\memeproject\\code\html\\memes\\' + name + ".js", 'w')
json.dump(toJSON(allThumbs, [meme[1] for meme in selectedMemes]), fp)
fp.close()

Refinement
==

1) Pick represenative thumbnaul

2) Compare all thumbnails with representative. 

3) Add new thumbnails to original list

In [226]:
# Pick thumbnail from existing json file
name = "Awkward_Penguin"
clickedList, memes = verifyMemes(name + ".js") # memes[clickedList[0]] returns the first clicked item
jsonmemes = json.load(open("D:\\memeproject\\code\\html\\memes\\" + name +".js",'r'))

Meme size is 5877


In [227]:
%%capture
# Compare thumbnail
loc = r"D:\\memeproject\\thumbnails\\" +  memes[clickedList[0]]["id"] + ".jpg"
(thumbList, a) = compareAll(loc, allThumbs)

In [228]:
# Look at all thumbs
jsonmemelist = [meme["id"] for meme in memes]
jsonmemeset  = OrderedSet(jsonmemelist)
alltset      = OrderedSet([thumb[1][0:-4] for thumb in thumbList])
diffset      = alltset - jsonmemeset
diffList     = [d + ".jpg" for d in diffset]
clickedList  = previewThumbs(diffList, (30,30))
show()

In [229]:
selectedMemes = itemgetter(*clickedList)(diffList)
concatMemes   = [meme + ".jpg" for meme in jsonmemelist] + list(selectedMemes)
assert(len(concatMemes) == len(list(selectedMemes)) + len(jsonmemelist))
assert(len(set(concatMemes)) == len(concatMemes))
print "Previous length ", len(jsonmemeset), " Current length ", len(concatMemes)
js = toJSON(allThumbs, concatMemes)

Previous length  5877  Current length  5882


In [230]:
fp = open('D:\\memeproject\\code\html\\memes\\' + name + '.js', 'w')
json.dump(js, fp)
fp.close()